In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
# from keras_bert import load_trained_model_from_checkpoint, Tokenizer
from bert_features.bert_features import BertEmbedding

import codecs
import unicodedata
import re
import numpy as np
import os
import io
import time

# import MeCab

In [2]:
tf.enable_eager_execution()
print(tf.__version__, tf.keras.__version__)

1.14.0 2.2.4-tf


In [3]:
# jpn-english material
path_to_file = "./jpn-eng/jpn.txt"

In [4]:
# bert model parameters
SEQ_LEN = 64
BATCH_SIZE = 64
OUTPUT_LAYER_NUM = 4
LEARNING_RATE = 1e-5

PRETRAINED_PATH = 'bert-master\\multi_cased_L-12_H-768_A-12'
CONFIG_PATH = os.path.join(PRETRAINED_PATH, 'bert_config.json')
CHECKPOINT_PATH = os.path.join(PRETRAINED_PATH, 'bert_model.ckpt')
VOCAB_PATH = os.path.join(PRETRAINED_PATH, 'vocab.txt')

In [5]:
# load bert model
# bert_model = load_trained_model_from_checkpoint(
#   CONFIG_PATH,
#   CHECKPOINT_PATH,
#   training=False,
#   trainable=False,
#   output_layer_num=OUTPUT_LAYER_NUM,
#   seq_len=SEQ_LEN
# )

In [6]:
# prepare token->idx dictionary
# def make_token_dict(vocab_path):
#   token_dict = {}
#   with codecs.open(vocab_path, 'r', 'utf8') as reader:
#     for line in reader:
#       if line != ' \n':
#         token = line.strip()
#       else:
#         token = line.strip('\n')
#       token_dict[token] = len(token_dict)
#   return token_dict

In [7]:
def preprocess_eng_sentence(w):
    w = w.lower().strip()

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '[CLS] ' + w + ' [SEP]'
    return w

def preprocess_jpn_sentence(w):
    m = MeCab.Tagger ("-Owakati")
    w = '[CLS] ' + m.parse(w).strip().strip('\n') + ' [SEP]'
    return w

In [8]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    eng, jpn = [], []
    for l in lines[:num_examples]:
        (eng_sentence, jpn_sentence) = l.split('\t')
        (eng_sentence, _) = l.split('\t')
        eng.append(preprocess_eng_sentence(eng_sentence))
        jpn.append(jpn_sentence)
    return eng, jpn

In [9]:
eng, jpn = create_dataset(path_to_file, None)

In [10]:
print(eng[3])
print(jpn[3])

[CLS] hi . [SEP]
やっほー。


In [11]:
def max_length(tensor):
    return max(len(t) for t in tensor)

def tokenize(lang):
    # convert words(of a sentence) into word indexes
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
                     filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)
    # padding word indexes(of sentences) to the same length(using maximum length of all sentences)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                           padding='post')

    return tensor, lang_tokenizer

# def load_dataset(path, num_examples=None):
#     # creating cleaned input, output pairs
#     targ_lang, inp_lang = create_dataset(path, num_examples)

#     input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
#     target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

#     return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [12]:
def load_dataset(path, num_examples=None, verbose=False):
  # creating cleaned input, output pairs
  targ_lang, inp_lang_raw = create_dataset(path, num_examples)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

#   input_token_dict = make_token_dict(input_vocab_path)
#   inp_lang_tokenizer = Tokenizer(input_token_dict, cased=True)
  
#   lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
#   inp_ids, inp_segments = [], []
#   for l in lines[:num_examples]:
#       (_, jpn_sentence) = l.split('\t')
#       id, segment = inp_lang_tokenizer.encode(jpn_sentence, max_len=SEQ_LEN)
#       inp_ids.append(id)
#       inp_segments.append(segment)
#       if verbose:
#         print('{}->{}{}'.format(jpn_sentence, id, segment))
  

  return inp_lang_raw, target_tensor, targ_lang_tokenizer

In [13]:
# inp_lang_raw, target_tensor, targ_lang = load_dataset(path_to_file, 3)

In [14]:
# Try experimenting with the size of that dataset
num_examples = 45000
# input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)
inp_lang_raw, target_tensor, targ_lang = load_dataset(path_to_file, num_examples)
input_tensor = []

# Calculate max_length of the target tensors
# max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)
max_length_targ, max_length_inp = max_length(target_tensor), SEQ_LEN

# Creating training and validation sets using an 80-20 split
inp_lang_raw_train, inp_lang_raw_val, target_tensor_train, target_tensor_val = train_test_split(inp_lang_raw, target_tensor, test_size=0.2)

# Show length
print(len(inp_lang_raw_train), len(target_tensor_train), len(inp_lang_raw_val), len(target_tensor_val))

36000 36000 9000 9000


In [15]:
print(max_length_targ, max_length_inp)

29 64


In [16]:
# def convert(lang, tensor):
#   if isinstance(lang, tf.keras.preprocessing.text.Tokenizer):
#     for t in tensor:
#       if t!=0:
#         print ("%d ----> %s" % (t, lang.index_word[t]))

# print ("Input Language; index to word mapping")
# convert(inp_lang, input_tensor_train[0][0])
# print ()
# print ("Target Language; index to word mapping")
# convert(targ_lang, target_tensor_train[0])

In [17]:
BUFFER_SIZE = len(inp_lang_raw_train)
BATCH_SIZE = 64
steps_per_epoch = len(inp_lang_raw_train)//BATCH_SIZE
embedding_dim = 768 # bert-base
units = 1024
# vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((inp_lang_raw_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [18]:
print(inp_lang_raw_train[:3])

['彼は歩くのが速い。', '私は歩いて家に帰らなければならなかった。', '搭乗券を拝見します。']


In [19]:
class BertFeaturesLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(BertFeaturesLayer, self).__init__()
        self.bertembedding = BertEmbedding()

    def build(self, input_shape):
        pass

    def call(self, input):
        result = self.bertembedding.get_embeddings(input)
        return result

In [20]:
class Encoder(tf.keras.Model):
    def __init__(self, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embeddings = BertFeaturesLayer()
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embeddings(x)
        output, state = self.gru(np.array(x), initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [21]:
encoder = Encoder(units, BATCH_SIZE)
# sample input
sample_hidden = encoder.initialize_hidden_state()
example_input_batch = inp_lang_raw_train[:64]

sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

W0902 14:53:57.027435 12908 deprecation_wrapper.py:119] From bert-master\modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0902 14:53:58.980288 12908 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0902 14:53:58.981276 12908 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x000002AFF99DD9D8>) includes params argument, but params are not passed to Estimator.
W0902 14:53:58.990267 12908 estimator.py:1811] Using temporary folder as model directory: C:\Users\di.sun\AppData\Local\Temp\tmphs_r35cw
W0902 14:53:58.993269 12908 tpu_context.py:211] eval_on_tpu ignored because use_

call estimator.predict


W0902 14:54:04.454303 12908 deprecation.py:323] From bert-master\modeling.py:671: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0902 14:54:07.798350 12908 deprecation_wrapper.py:119] From C:\Users\di.sun\Box\disun\jupyter notes\tensorflow\bert_features\bert_features.py:124: The name tf.trainable_variables is deprecated. Please use tf.compat.v1.trainable_variables instead.

W0902 14:54:07.844346 12908 deprecation_wrapper.py:119] From C:\Users\di.sun\Box\disun\jupyter notes\tensorflow\bert_features\bert_features.py:137: The name tf.train.init_from_checkpoint is deprecated. Please use tf.compat.v1.train.init_from_checkpoint instead.

W0902 14:54:09.466362 12908 deprecation.py:323] From C:\Users\di.sun\AppData\Local\Continuum\anaconda3\envs\tensorflow1\lib\site-packages\tensorflow\python\ops\array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.arra

Encoder output shape: (batch size, sequence length, units) (64, 64, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [22]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [23]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 64, 1)


In [24]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

In [25]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((64, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 8844)


In [26]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [27]:
checkpoint_dir = './training_with_bert_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt-45k-30epoch")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [28]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([targ_lang.word_index['[cls]']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [ ]:
loss_test = train_step(inp_lang_raw_train[:64], target_tensor[:64], encoder.initialize_hidden_state())
print(loss_test)

In [29]:
iterates = len(inp_lang_raw_train) // BATCH_SIZE
# for batch in range(iterates):
#     print("{}-{}".format(batch * 64, (batch+1) * 64))

In [30]:
EPOCHS = 4

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    iterates = len(inp_lang_raw_train) // BATCH_SIZE
    for batch in range(iterates):
        inp = inp_lang_raw_train[batch * 64:(batch+1) * 64]
        targ = target_tensor[batch * 64:(batch+1) * 64]
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss        
#     for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
#         batch_loss = train_step(inp, targ, enc_hidden)
#         total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

call estimator.predict
call estimator.predict
Epoch 1 Batch 0 Loss 1.0783
call estimator.predict
call estimator.predict


KeyboardInterrupt: 

In [ ]:
EPOCHS = 4

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [ ]:
def evaluate(sentence, verbose=False):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    if verbose == True:
        print('Original sentence: {}'.format(sentence))

    sentence = preprocess_jpn_sentence(sentence)
    if verbose == True:
        print('Preprocessed sentence: {}'.format(sentence))

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    if verbose == True:
        print('Word sequences of sentence: {}'.format(inputs))
        
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    if verbose == True:
        print('Paded sequences of sentence: {}'.format(inputs))

    inputs = tf.convert_to_tensor(inputs)
    if verbose == True:
        print('Tensor of sentence: {}'.format(inputs))

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    if verbose == True:
        print('enc_out: {}'.format(enc_out))
        print('enc_hidden: {}'.format(enc_hidden))

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)
    if verbose == True:
        print('dec_input: {}'.format(dec_input))

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)
        if verbose == True:
            print('dec_input: {}'.format(dec_input))

    return result, sentence, attention_plot

In [ ]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [ ]:
def translate(sentence, verbose=False):
    result, sentence, attention_plot = evaluate(sentence, verbose)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
translate(u'雨が嫌い。')

In [ ]:
translate('いい天気ですね。', verbose=True)